In [ ]:
import os
import json

In [ ]:
def ph2dt(
    node_i: int,
    config_json,
    hypodd_phase,
    station_dat,
    ct_file,
    hypodd_event,
    data_path: str,
)->int:

    import json
    import os
    from datetime import datetime

    with open(config_json, "r") as fp:
        config = json.load(fp)

    hypodd_path = os.path.join(data_path, "hypodd")
    if not os.path.exists(hypodd_path):
        os.mkdir(hypodd_path)

    # try:
    #     minioClient = Minio(s3_url, access_key='minio', secret_key='minio123', secure=secure)
    #     minioClient.fget_object(bucket_name, f"{config['region']}/hypodd_{node_i:03d}.pha", os.path.join(hypodd_path, f"hypodd_{node_i:03d}.pha"))
    # except Exception as err:
    #     print(f"ERROR: can not access minio service! \n{err}")
    #     pass

    ph2dt = """* ph2dt.inp - input control file for program ph2dt
* Input station file:
stations.dat
* Input phase file:
hypodd.pha
*MINWGHT: min. pick weight allowed [0]
*MAXDIST: max. distance in km between event pair and stations [200]
*MAXSEP: max. hypocentral separation in km [10]
*MAXNGH: max. number of neighbors per event [10]
*MINLNK: min. number of links required to define a neighbor [8]
*MINOBS: min. number of links per pair saved [8]
*MAXOBS: max. number of links per pair saved [20]
*MINWGHT MAXDIST MAXSEP MAXNGH MINLNK MINOBS MAXOBS
   0      120     10     50     8      8     100
"""

    with open(os.path.join(hypodd_path, "ph2dt.inp"), "w") as fp:
        fp.writelines(ph2dt)

    def copy_file(fp_from, fp_to):
        with open(fp_from, "r") as fp:
            lines = fp.readlines()
        with open(fp_to, "w") as fp:
            fp.writelines(lines)

    copy_file(hypodd_phase, os.path.join(hypodd_path, "hypodd.pha"))
    copy_file(station_dat, os.path.join(hypodd_path, "stations.dat"))

    PH2DT_CMD = f"cd {hypodd_path} && /opt/hypodd/ph2dt ph2dt.inp"
    print(PH2DT_CMD)
    if os.system(PH2DT_CMD) != 0:
        raise (f"{PH2DT_CMD}" + " failed!")

    copy_file(os.path.join(hypodd_path, "dt.ct"), ct_file)
    copy_file(os.path.join(hypodd_path, "event.sel"), hypodd_event)

    ## upload to s3 bucket
    # try:
    #     from minio import Minio

    #     minioClient = Minio(s3_url, access_key="minio", secret_key="minio123", secure=secure)
    #     if not minioClient.bucket_exists(bucket_name):
    #         minioClient.make_bucket(bucket_name)

    #     minioClient.fput_object(
    #         bucket_name,
    #         f"{config['region']}/hypodd/dt_{node_i:03d}.ct",
    #         ct_file,
    #     )

    #     minioClient.fput_object(
    #         bucket_name,
    #         f"{config['region']}/hypodd/event_{node_i:03d}.sel",
    #         hypodd_event,
    #     )

    # except Exception as err:
    #     print(f"ERROR: can not access minio service! \n{err}")
    #     pass

    # try:
    #     minioClient.fput_object(
    #         bucket_name,
    #         f"{config['region']}/dt_{node_i:03d}.ct",
    #         f"{os.path.join(hypodd_path, 'dt.ct')}",
    #     )
    #     minioClient.fput_object(
    #         bucket_name,
    #         f"{config['region']}/event_{node_i:03d}.dat",
    #         f"{os.path.join(hypodd_path, 'event.dat')}",
    #     )
    #     minioClient.fput_object(
    #         bucket_name,
    #         f"{config['region']}/event_{node_i:03d}.sel",
    #         f"{os.path.join(hypodd_path, 'event.sel')}",
    #     )
    # except Exception as err:
    #     print(f"ERROR: can not access minio service! \n{err}")
    #     pass

    return 0

In [ ]:
# デバッグ: HypoDDの実行ファイル確認
print("Checking HypoDD executables:")
print(f"ph2dt exists: {os.path.exists('/opt/hypodd/ph2dt')}")
print(f"hypoDD exists: {os.path.exists('/opt/hypodd/hypoDD')}")

# 実行権限確認
import stat
if os.path.exists('/opt/hypodd/ph2dt'):
    st = os.stat('/opt/hypodd/ph2dt')
    print(f"ph2dt permissions: {oct(st.st_mode)}")

In [ ]:
ph2dt(
    0,
    "config/config.json",
    "hypodd/hypodd_phase.pha",
    "hypodd/stations.dat",
    "hypodd/dt.ct",
    "hypodd/event.sel",
    "./",
)

In [8]:
# Kubeflow Pipelines UI用のメタデータ出力
if os.environ.get('ELYRA_RUNTIME_ENV') == 'kfp':
    # For information about Elyra environment variables refer to
    # https://elyra.readthedocs.io/en/stable/user_guide/best-practices-file-based-nodes.html#proprietary-environment-variables

    metadata = {
        'outputs': [
            {
                'storage': 'inline',
                'source': f'# Configuration Setup Complete\n...',
                'type': 'markdown',
            }
        ]
    }

    with open('mlpipeline-ui-metadata.json', 'w') as f:
        json.dump(metadata, f)